# Видобуток та збагачення структурованих даних

### ТЕОРЕТИЧНА ЧАСТИНА ТА ПРИКЛАДИ

Для роботи з структурованими табличними даними, які розміщені в реляційній БД [SQLite](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiUyMOv1orvAhWDK3cKHac9DyMQFjAAegQIBxAD&url=https%3A%2F%2Fru.wikipedia.org%2Fwiki%2FSQLite&usg=AOvVaw1s-RMK2VC2tXyDZxZXDYDo) в Python передбачена стандартна бібліотека [sqlite3](https://docs.python.org/3/library/sqlite3.html)

In [ ]:
# підключення бібліотеки
import sqlite3

#### ПРИКЛАД ВИЛУЧЕННЯ ДАНИХ З БД
Створити БД SQLite з ім'ям `FIT-4m`, та імпорувати в неї таблицю, що містить журнал групи `jornal`

In [ ]:
# створюємо з'єднання з БД
conn = sqlite3.connect("FIT-4m.db")

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

In [ ]:
# створюємо об'єкт `cursor`, що відповідає за реалізацію операцій з таблицями БД
cur = conn.cursor()

In [ ]:
print(type(cur), end='\n\n')
print(dir(cur))

##### виведемо ВСІ записи таблиці `jornal`

In [ ]:
# створимо інстанс для операції `SELECT`
students_cur = cur.execute("SELECT * FROM jornal")

# будуємо список записів за допомогою метода `fetchall()`
students_list = students_cur.fetchall()

_ = [print(x) for x in students_list]

#### ПРИКЛАД ЗБАГАЧЕННЯ ДАНИХ З ЗОВНІШНЬОГО ДЖЕРЕЛА

Додати до списка студентів їх оцінки за вступне тестування, які розташовані в csv файлі `test_result.csv`

імпортуємо `test_result.csv` в робочу БД як таблицю `test`

[імпорт csv файла в sqlite через CLI](https://www.sqlitetutorial.net/sqlite-import-csv/) 

##### Cтворимо нову таблицю, яка буде містити вміст таблиці `jornal` та поле оцінок з таблиці `test`



In [ ]:
# створимо вираз на побудову пустої таблиці `jornal_test` з відповідними полями
sql_stmt = "CREATE TABLE IF NOT EXISTS jornal_test \
              (Surname TEXT,         \
               Name TEXT,            \
               Email TEXT,           \
               GitHub TEXT,          \
               Test_result INT);"

# виконаемо запит на створення таблиці
cur.execute(sql_stmt)

In [ ]:
# перевіримо стан виконання операції - виведемо список таблиць
tables_list = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables_list.fetchall()

In [ ]:
# створимо запит на об'єднання таблиць 'jornal' та 'test' по прізвищу студента
sql_stmt = "SELECT jornal.*, test_result.test FROM  \
            jornal LEFT JOIN test_result ON jornal.Surname = test_result.Surname "

In [ ]:
# для перевірки виконаємо і виведемо результати запиту
cur.execute(sql_stmt).fetchmany(7)

In [ ]:
# заповнимо пусту таблицю результатми  запиту на злиття
_ = cur.execute("INSERT INTO jornal_test " + sql_stmt)

In [ ]:
# збережемо вміст нової таблиці в список
jornal_test_list = cur.execute("SELECT * FROM jornal_test").fetchall()

_ = [print(x) for x in jornal_test_list]

In [ ]:
# завершимо всі транзакції
conn.commit()

# закріємо з'єднання
conn.close()

#### ПРИКЛАД  ДЕСКРИПТИВНОГО АНАЛІЗУ ДАНИХ

Провести [попередній аналіз](https://ru.wikipedia.org/wiki/Описательная_статистика) отриманих даних з ціллю виявленя відхилень, помилок та інших непридатних даних.

Сведемо результати аналізу в таблицю, яка має наступний вигляд:

№| Показчик | Значення
:--:|:-------|-------:
1| кількість спостережень | xx 
2| кількість пустих значень | xx
4| середній бал|  xx.x
5| максимальний бал | xx
6| мінімальний бал  | xx
7| стандартне відхилення | xx.x
8| розмах вариації  | xx


In [ ]:
# підключимо бібліотеку 'numpy' і дамо їй аліас 'np'
import numpy as np

ознайомитись з призначенням, можливостями та основним функціями [бібліотеки numpy](https://numpy.org) 

In [ ]:
# перетворимо список студетів в numpay матрицю
jornal_test_arr = np.array(jornal_test_list)

In [ ]:
print(type (jornal_test_arr), jornal_test_arr.shape)

In [ ]:
# створимо масив с результатами тестів 
test_result = np.delete(jornal_test_arr, [1,2,3], axis=1)

In [ ]:
# Значення 'None' - не є числом, то заміним його на 0
test_result[test_result == None] = 0

In [ ]:
# залишити тільки статистично важливі показчики
test_result = np.delete(test_result, 0, axis=1)

In [1]:
# будуємо словник з вихідними розрахунками
result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)])
    }

NameError: name 'test_result' is not defined

In [3]:
result_dict

NameError: name 'result_dict' is not defined

In [10]:
# вивести шапку
print(
"""
=======================================
№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     
=======================================
"""      
      )
# вивести результати аналізу датасета
i = 1
for key, value in result_dict.items():
    print (f'{i:<3} {key:<25}  {value}')
    i += 1


№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     



NameError: name 'result_dict' is not defined

### ІНДИВІДУАЛЬНЕ ЗАВДАННЯ

Користуючись результатами, що отримані в [Лабораторній роботі № 3](https://shkliarskiy.moodlecloud.com/mod/page/view.php?id=1193) виконати процедури видобутку, збагачення та попереднього аналізу даних.

__Постановка__: В 3-й лабораторній роботі отримано показчик, що характеризує окрему властивість квартири (ціна, метраж та ін.). 
Необхідно:
1. Відобразити цей показчик на адресу квартири, яка знаходиться в файлі `street_name.csv`, тобто побудувати новий файл `street_flat_propery.csv` якій містить 2 колонки: _<назва вулиці> <ваш показчик>_
2. На основі цьго файлу зробити відповідну таблицю в _SQLite_ та збагатити цю дані в цій таблиці назвою району де розташована квартира (дані по районах додаються) 
3. Провести попередній аналіз отриманих даних - вивести первинні статистики показчика по районах (_середне_ , _мінімальне_ , _максимальне_ і т.д. - див. приклад вище)

In [3]:
# Злиття файлу з показчиком та файлу з назвами вулиць в новий файл
import os
import numpy as np

path = 'E:\Lab4\Lab3'
filenames = []

for root, dirs, files in os.walk(path): 
    for file in files:  
        filenames.append(path +  "\\" + file)

with open('E:\Lab4\street_name.csv', 'r', encoding='utf-8') as f1, \
    open('E:\Lab4\street_flat_propery.csv', 'w') as f3:     
    f1_lines = f1.readlines()
    for line_index in range (0, len(f1_lines)):     
        element = f1_lines[line_index].split()   

        if(not element or element[0] == '***'):
            result = "No exist" + ","
        else:
            f1_splited = f1_lines[line_index].split()
            result = f1_splited[0] + "," 

        for sname in filenames:
            with open(sname) as f2:
                f2_lines = f2.readlines()
                f2_splited = f2_lines[line_index].split()

                result+= f2_splited[0] + ","
                                                
        f3.write(result[:-1] + '\n')
        jornal_test_list = cur.execute("SELECT * from street_flat_propery").fetchall()
jornal_test_arr = np.array(jornal_test_list)

x_result = np.delete(jornal_test_arr, [0], axis=1)

length = x_result.shape

for j in range(0, length[1]):
    test_result = x_result[:,j].astype(float)
    
    result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)]) 
    }
    
    columns_names = cur.execute("PRAGMA table_info('street_flat_propery')").fetchall()
    print("Назва таблиці: " + columns_names[j][1])
    
    print(
"""
=======================================
№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     
=======================================
"""      
      )

    i = 1
    for key, value in result_dict.items():
        print (f'{i:<3} {key:<25}  {value}')
        i += 1
    print('\n')


UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 175: character maps to <undefined>

#### імпортувати в БД отриманий файл 'street_flat_propery.csv' через CLI
- __sqlite3 districts.db__ - запустити SQLite та відкрити БД
- __.mode csv__ - перевести БД для роботи з csv-файлами
- __.import street_flat_propery.csv street_flat_propery__ - створює таблицю _street_flat_propery_ та завантажує в неї вміст файлу _street_flat_propery.csv_
- __SELECT * FROM street_flat_propery LIMIT 10;__ - вивести 10 записів для перевірки
- __.quit__ - завершити роботу з CLI SQLite



In [ ]:
# Підключитись до БД 'districts.db' в якій є прив'язка вулиць до районів 
import sqlite3
conn = sqlite3.connect("districts.db")


# створити відповідний курсор
cur = conn.cursor()
streets_cur = cur.execute("SELECT * FROM street_district")

In [ ]:
# для перевірки виведіть 10 записів з таблиці 'street_district'
streets_cur = cur.execute("SELECT * FROM street_district")

streets_list = streets_cur.fetchmany(10)

_ = [print(street) for street in streets_list]

In [ ]:
# побудувати запит на створення таблиці 'result' з 3-ма полями:
sql_stmt = "CREATE TABLE IF NOT EXISTS result \
              (district TEXT,         \
               street TEXT,            \
               property INT);"


In [ ]:
# виконати запит
cur.execute(sql_stmt)
tables_list = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables_list.fetchall()

In [ ]:
# перевірити схему таблиці
cur.execute ("SELECT name FROM PRAGMA_TABLE_INFO('result');").fetchall()

In [ ]:
# створити запит на злиття таблиці 'street_flat_propery' та 'street_district' 
# по полю 'street'
stmt_join = "SELECT street_flat_propery.*, street_district.type, street_district.district FROM   \
            street_flat_propery LEFT JOIN street_district ON street_flat_propery.street = street_district.street "

In [ ]:
# перевірити запит
cur.execute(stmt_join).fetchall()

In [ ]:
# створити запит для наповнення таблиці `result` результатом запиту на об'єднання
stmt_insert = "result:  " + stmt_join

In [ ]:
# виконати запит на заповнення
cur.execute(stmt_insert)
conn.commit()
conn.close()

In [ ]:
# сберегти вміст створеної таблиці у вигляді списка
import sqlite3
conn = sqlite3.connect("districts.db")
cur = conn.cursor()
result_list = cur.execute("SELECT * FROM street_result").fetchall()

OperationalError: no such table: street_result

#### для аналізу скористатися бібліотекою NumPy

In [ ]:
import numpy as np

In [ ]:
# створити np-масив з списку `result_list`
jornal_list = cur.execute("SELECT * from street_result").fetchall()
jornal_arr = np.array(jornal_list)
x_result = np.delete(jornal_arr, [0,9,10], axis=1)
print(x_result)
[[21 37 1575.0 ... 25 2015 None]
 [11 58 680.0 ... 25 0 None]
 [5 56 1400.0 ... 18 2011 None]
 ...
 [19 74 1461.0 ... 20 1995 None]
 [11 17.9 1019.0 ... 25 0 None]
 [7 18.53 2263.0 ... 11 0 'Лісовий, Чернігівський']]

SyntaxError: invalid syntax (1026647659.py, line 7)

In [ ]:
# побудувати словник для результатів аналізу
result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]),
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)]) 
    }

In [16]:
disctricts_list = cur.execute("SELECT DISTINCT district FROM street_result").fetchall()
for i in range(len(disctricts_list)):
    values = disctricts_list[i]
    district = values[0]   
    print("Назва району: " + district + '\n')
    jornal_test_list = cur.execute("SELECT * from street_result WHERE district = ?", (district,)).fetchall()    
    jornal_test_arr = np.array(jornal_test_list)
    
    if(len(jornal_test_arr) == 0):
        continue
        
    x_result = np.delete(jornal_test_arr, [0,9,10], axis=1)
    
    length = x_result.shape
    columns_count = length[1]
    
    for j in range(0, length[1]):
        test_result = x_result[:,j].astype(float)        
        try:
            result_dict = {
    "кількість спостережень"   : len(test_result),
    "кількість пустих значень" : len(test_result) - np.count_nonzero(test_result),
    "середній бал"             : round(np.mean(test_result),1),
    "максимальний бал"         : np.max(test_result),
    "мінімальний бал"          : np.min(test_result[np.nonzero(test_result)]) ,
    "стандартне відхилення"    : round(np.std(test_result),1),
    "розмах вариації"          : np.max(test_result) - np.min(test_result[np.nonzero(test_result)]) 
    }
        
        except ValueError: "Empty"
        pass
        columns_names = cur.execute("PRAGMA table_info('street_flat_propery')").fetchall()
        print("Назва поля: " + columns_names[j + 1][1])
        
        print(
"""
=======================================
№  : ПОКАЗЧИК               : ЗНАЧЕННЯ     
=======================================
"""   )
       

OperationalError: no such table: street_result